In [23]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def box2UTMh(x, y, x0, y0, theta):
    '''
    2D rotation and translation of x, y
    Input:
        x, y - row vectors of original coordinates (must be same size)
        x0, y0 - Offset (location of x, y = (0,0) in new coordinate system)
        theta - Angle of rotation (degrees, CCW from x-axis == Cartesian coorinates)
    Returns:
        xr, yr - rotated, offset coordinates
    '''
    thetar = np.radians(theta)
    c, s = np.cos(thetar), np.sin(thetar)

    # homogenous rotation matrix
    Rh = np.array(((c, -s,  0.),\
                   (s,  c,  0.),\
                   (0., 0., 1.)))
    # homogenous translation matrix
    Th = np.array(((1., 0., x0),\
                   (0., 1., y0),\
                   (0., 0., 1.)))

    # homogenous input x,y
    xyh = np.vstack((x,y,np.ones_like(x)))

    # perform rotation and translation
    xyrh=np.matmul(np.matmul(Th,Rh),xyh)
    xr = xyrh[0,:]
    yr = xyrh[1,:]
    return xr, yr

def pcoord(x, y):
    """
    Convert x, y to polar coordinates r, az (geographic convention)
    r,az = pcoord(x, y)
    """
    r  = np.sqrt( x**2 + y**2 )
    az=np.degrees( np.arctan2(x, y) )
    # az[where(az<0.)[0]] += 360.
    az = (az+360.)%360.
    return r, az

def xycoord(r, az):
    """
    Convert r, az [degrees, geographic convention] to rectangular coordinates
    x,y = xycoord(r, az)
    """
    x = r * np.sin(np.radians(az))
    y = r * np.cos(np.radians(az))
    return x, y

def UTM2Island(eutm, nutm, eoff=378489.45785127, noff=3855740.50113774, rot=42.):
    """
    Convert UTM NAD83 Zone 18N easting, northing to N. Core Banks alongshore, cross-shore coordinates
    xisl, yisl = UTM2Island( eutm, nutm )
    """
    [r,az]=pcoord(eutm-eoff,nutm-noff)
    az = az+rot;
    [xisl,yisl]=xycoord(r,az)
    return xisl, yisl

def UTM2rot(xutm,yutm,r):
    """
    Convert UTM coordinates to rotated coordinates
    """
    # Convert origin to UTM
    xu,yu = box2UTMh(0.,0.,r['e0'],r['n0'],r['theta'])
    # reverse the calc to find the origin (UTM =0,0) in box coordinates.
    # First, just do the rotation to see where Box = 0,0 falls
    print('Location of box(0,0) in UTM coords: ',(xu,yu))
    xb0,yb0 = box2UTMh(xu,yu,0.,0.,-r['theta'])
    # Then put in negative values for the offset
    #TODO: why does this return a list of arrays?
    print('xb0, yb0:',xb0,yb0)
    xbl,ybl = box2UTMh(xutm,yutm,-xb0,-yb0,-r['theta'])
    # this fixes it...probably should fix box2UTMh
    xb = np.concatenate(xbl).ravel()
    yb = np.concatenate(ybl).ravel()
    return xb, yb

eutm =  378489.45785126585
nutm = 3855740.5011377367+100.
r = {'name':'ncorebx_v5','e0':378489.45785126585,'n0': 3855740.5011377367,'xlen': 36650.0,'ylen': 1500.0,'dxdy': 1.0,'theta': 42.0}

xi,yi = UTM2rot(eutm,nutm,r)
print('xi, yi',xi,yi)
noff = 3855740.50113774
eoff = 378489.45785127
rot = 42.
[r,az]=pcoord(eutm-eoff,nutm-noff)
print('r,az: ',r,az)
az = az+rot;
[xisl,yisl]=xycoord(r,az);
print(xisl,yisl)

print( UTM2Island(eutm,nutm))

Location of box(0,0) in UTM coords:  (array([378489.45785127]), array([3855740.50113774]))
xb0, yb0: [2861266.46158861] [2612114.72137167]
xi, yi [66.91306064] [74.31448255]
r,az:  99.99999999674037 359.99999999763213
66.91306063063347 74.3144825480824
(66.91306063063347, 74.3144825480824)
